This is the main Sensor Kennel application that will 
identify, read and write any sensor that's plugged in.

On the lower side:  
  Pin1 (left) Gnd, Pin3 3v3, Pin-3 (from right) SDA, Pin-5 SCL.
  
Do we want to draw a graph of numbers if a button is pressed?

Should we have MQTT numbers going out from it?

Or just keep it comprehensive

In [974]:
%serialconnect

Connecting to --port=/dev/ttyUSB2 --baud=115200 
Ready.


In [209]:
%sendtofile --binary --source OLED_driver.py
%sendtofile --quiet --source device_detection.py
%sendtofile --quiet --source SI7021_funcs.py
%sendtofile --quiet --source TSL561_funcs.py
%sendtofile --quiet --source VL53L0X_funcs.py
%sendtofile --quiet --source VL6180_funcs.py
%sendtofile --quiet --source SDOF_funcs.py
%sendtofile --quiet --source SHT31D_funcs.py
%sendtofile --quiet --source BME280_funcs.py
%sendtofile --quiet --source BNO055serial_funcs.py


Sent 55 lines (2042 bytes) to SI7021_funcs.py.
Sent 54 lines (1757 bytes) to TSL561_funcs.py.
Sent 42 lines (1800 bytes) to SDOF_funcs.py.
Sent 55 lines (1614 bytes) to device_detection.py.
Sent 3084 bytes in 103 chunks to OLED_driver.py.
Sent 88 lines (2732 bytes) to BNO055serial_funcs.py.


In [975]:
%sendtofile main.py

# connect to the OLED and initialize (and get its i2c bus)
from OLED_driver import i2c, fbuff, oledshow, doublepixels, fatntext
import machine, time
time.sleep(0.5)

from device_detection import IdentifyI2CDevice, IdentifyUARTDevice

devices = IdentifyI2CDevice(i2c)
if not devices:
    uart = machine.UART(1, baudrate=9600, rx=13, tx=17)
    devices = IdentifyUARTDevice(uart)
if not devices:
    devices.append("nothing found")

# this might look like a lot of code, but it's the complete 
# visual output of text and numbers that we do per sensor here
for dev in devices:
    devname, devdesc = dev.split(" ", 1)
    fbuff.fill(0)
    fbuff.text(devname, 0, 0, 1)
    fbuff.text(devdesc, 0, 12, 1)
    doublepixels()
    oledshow()
    time.sleep(2)
    
if devname == "SI7021":
    from SI7021_funcs import setupSI7021, SI7021humiditytemp
    setupSI7021(i2c)
    while True:
        h, t = SI7021humiditytemp()
        fbuff.fill(0)
        fbuff.text("SI7021", 0, 0, 1)
        fbuff.text("humid:", 0, 23, 1)
        fatntext("%.1f"%h, 50, 16)
        fbuff.text("temp:", 0, 53, 1)
        fatntext("%.1f"%t, 50, 46)
        oledshow()

if devname == "VL53L0X":
    from VL53L0X_funcs import VL53L0Xinit, VL53L0Xdist
    VL53L0Xinit(i2c)
    while True:
        d = VL53L0Xdist()
        fbuff.fill(0)
        fbuff.text("VL53L0X", 0, 0, 1)
        fbuff.text("dist(mm):", 8, 20, 1)
        fatntext("%d" % d, 40, 36)
        oledshow()
        
if devname == "VL6180":
    from VL6180_funcs import VL6180init, distmm
    VL6180init(i2c)
    while True:
        d = distmm()
        fbuff.fill(0)
        fbuff.text("VL53L0X", 0, 0, 1)
        fbuff.text("dist(mm):", 8, 20, 1)
        fatntext("%d" % d, 40, 36)
        oledshow()

if devname == "TSL561":
    from TSL561_funcs import setupTSL561, luminosity, setregtimings
    setupTSL561(i2c)
    setregtimings(True, 1)
    while True:
        br, ir = luminosity()
        fbuff.fill(0)
        fbuff.text("TSL561", 0, 0, 1)
        fbuff.text("broad:", 0, 23, 1)
        fatntext("%d"%br, 50, 16)
        fbuff.text("ir:", 0, 53, 1)
        fatntext("%d"%ir, 50, 46)
        oledshow()
        time.sleep(0.1)

if devname == "TMP102":
    while True:
        r = i2c.readfrom_mem(0x48, 0x00, 2)
        t = r[0] + r[1]/256
        fbuff.fill(0)
        fbuff.text("TMP102", 0, 0, 1)
        fbuff.text("temp:", 0, 23, 1)
        fatntext("%.03f"%t, 30, 40)
        oledshow()
        time.sleep(0.1)

if devname == "SDOF":
    from SDOF_funcs import SetupAccGyrMag, readvectorsensor
    SetupAccGyrMag(i2c)
    while True:
        fbuff.fill(0)
        x, y, z = readvectorsensor("a")
        fbuff.text("Acc:", 0, 0, 1)
        fbuff.text("%d"%x, 8, 8, 1)
        fbuff.text("%d"%y, 8, 16, 1)
        fbuff.text("%d"%z, 8, 24, 1)
        x, y, z = readvectorsensor("g")
        fbuff.text("gyr:", 64, 0, 1)
        fbuff.text("%d"%x, 72, 8, 1)
        fbuff.text("%d"%y, 72, 16, 1)
        fbuff.text("%d"%z, 72, 24, 1)
        x, y, z = readvectorsensor("c")
        fbuff.text("mag:", 40, 32, 1)
        fbuff.text("%d"%x, 48, 40, 1)
        fbuff.text("%d"%y, 48, 48, 1)
        fbuff.text("%d"%z, 48, 56, 1)
        oledshow()
        time.sleep(0.1)
        
if devname == "BME280":
    from BME280_funcs import bme280init, readBME280
    bme280init(i2c)
    while True:
        temp, pressure, humid = readBME280()
        fbuff.fill(0)
        fbuff.fill(0)
        fbuff.text("BME280", 0, 0, 1)
        fbuff.text("tmp:", 0, 16, 1)
        fatntext("%.2f"%(temp), 32, 10)
        fbuff.text("prs:", 0, 36, 1)
        fatntext("%.2f"%(pressure), 32, 30)
        fbuff.text("hum:", 0, 54, 1)
        fatntext("%.2f"%(humid), 32, 50)
        oledshow()
        time.sleep(0.11)

if devname == "SHT31D":
    from SHT31D_funcs import SHT31Dsetup, readSHT31D
    SHT31Dsetup(i2c)
    while True:
        tmp, humid = readSHT31D()
        fbuff.fill(0)
        fbuff.text("SHT31-D", 0, 0, 1)
        fbuff.text("temperature:", 10, 10, 1)
        fatntext("%.2f"%tmp, 20, 22)
        fbuff.text("humidity:", 10, 40, 1)
        fatntext("%.2f"%humid, 20, 50)
        oledshow()
        time.sleep(0.1)
        
        
if devname == "BNO055":
    from BNO055serial_funcs import InitBNO055, BNO055absorientation, BNO055calibstat
    InitBNO055(uart)
    while True:
        fbuff.fill(0)
        calibstat = None
        try:
            pitch, roll, orient = BNO055absorientation()
            calibstat = BNO055calibstat()
        except Exception as e:
            fbuff.text("BNO055", 0, 0, 1)
            fbuff.text(e.args[0], 0, 16, 1)
        if calibstat is not None:  
            fbuff.text("BNO055 :{0:08b}".format(calibstat), 0, 0, 1)
            fbuff.text("pitch:", 8, 16, 1)
            fatntext("%d"%int(pitch), 64, 10)
            fbuff.text("roll:", 16, 32, 1)
            fatntext("%d"%int(roll), 64, 28)
            fbuff.text("orient:", 0, 52, 1)
            fatntext("%d"%int(orient), 64, 48)
        oledshow()
        time.sleep(0.1)        

        
# read any GPS serial signals on 
import machine
uart.init(baudrate=9600)
while True:
    x = uart.readline()
    if x is not None:
        fbuff.fill(0)
        fbuff.text("GPS serial", 0, 0, 1)
        x = x.strip()
        for i in range(len(x)//16 + 1):
            fbuff.text(x[i*16:i*16+16], 0, 15+i*8, 1)
        oledshow()
        time.sleep_ms(100)
    time.sleep_ms(50)


Sent 179 lines (5417 bytes) to main.py.


In [968]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB2 --baud=115200 
Ready.


In [904]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


In [874]:
import machine
import os
print(os.listdir())

['boot.py', 'SI7021_funcs.py', 'TSL561_funcs.py', 'SDOF_funcs.py', 'device_detection.py', 'OLED_driver.py', 'BNO055serial_funcs.py', 'dmain.py']


In [875]:
from OLED_driver import i2c, fbuff, oledshow, doublepixels, fatntext


In [876]:
help(machine)

object <module 'umachine'> is of type module
  __name__ -- umachine
  mem8 -- <8-bit memory>
  mem16 -- <16-bit memory>
  mem32 -- <32-bit memory>
  freq -- <function>
  reset -- <function>
  unique_id -- <function>
  sleep -- <function>
  deepsleep -- <function>
  idle -- <function>
  disable_irq -- <function>
  enable_irq -- <function>
  time_pulse_us -- <function>
  Timer -- <class 'Timer'>
  WDT -- <class 'WDT'>
  SLEEP -- 2
  DEEPSLEEP -- 4
  Pin -- <class 'Pin'>
  Signal -- <class 'Signal'>
  TouchPad -- <class 'TouchPad'>
  ADC -- <class 'ADC'>
  DAC -- <class 'DAC'>
  I2C -- <class 'I2C'>
  PWM -- <class 'PWM'>
  RTC -- <class 'RTC'>
  SPI -- <class 'SoftSPI'>
  UART -- <class 'UART'>
  reset_cause -- <function>
  HARD_RESET -- 2
  PWRON_RESET -- 1
  WDT_RESET -- 3
  DEEPSLEEP_RESET -- 4
  SOFT_RESET -- 5
  wake_reason -- <function>
  PIN_WAKE -- 1
  EXT0_WAKE -- 1
  EXT1_WAKE -- 2
  TIMER_WAKE -- 3
  TOUCHPAD_WAKE -- 4
  ULP_WAKE -- 5


In [880]:
x = machine.ADC(machine.Pin(36))
help(x)

object ADC(Pin(36)) is of type ADC
  read -- <function>
  atten -- <function>
  width -- <classmethod>
  ATTN_0DB -- 0
  ATTN_2_5DB -- 1
  ATTN_6DB -- 2
  ATTN_11DB -- 3
  WIDTH_9BIT -- 0
  WIDTH_10BIT -- 1
  WIDTH_11BIT -- 2
  WIDTH_12BIT -- 3


In [890]:
x.atten(x.ATTN_0DB)
x.width(x.WIDTH_12BIT)
print(x.read())

1281


In [903]:
%sendtofile main.py

from OLED_driver import i2c, fbuff, oledshow, doublepixels, fatntext
import time, machine

x = machine.ADC(machine.Pin(36))
while True:
    fbuff.fill(0)
    fbuff.text("Pin36 ADC", 56, 56, 1)
    x.atten(x.ATTN_0DB)
    fatntext(str(x.read()), 0, 0)
    x.atten(x.ATTN_6DB)
    fatntext(str(x.read()), 0, 20)
    x.atten(x.ATTN_11DB)
    fatntext(str(x.read()), 0, 40)
    oledshow()
    time.sleep(0.1)

Sent 15 lines (404 bytes) to main.py.
